In [71]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt

from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity, Statevector
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT

import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

from oft import oft
from tools.classical import find_ideal_heisenberg, trotter_heisenberg_qutip, BitHandler
from op_fourier_trafo_unitary import brute_prepare_gaussian_state
from tools.quantum import *
from tools.classical import HamHam

np.random.seed(667)

In [72]:
def energy_state_for_bits(bitstring: str) -> np.ndarray:
    """Convert a bitstring to an energy eigenstate."""
    tensor_list = []
    for bit in bitstring:
        if bit == '0':
            tensor_list.append(qt.basis(2, 0))
        else:
            tensor_list.append(qt.basis(2, 1))

    state = qt.tensor(tensor_list).full()
    return state / np.linalg.norm(state)

def projector_to_energy_eigenspace(bitstring: str) -> np.ndarray:
    """Return the projector to the energy eigenstate corresponding to a bitstring."""
    state = energy_state_for_bits(bitstring)
    return state @ state.conj().T

### We shift every unit onto the energy, i.e. $\omega_0 = \frac{2\pi}{N}, t_0 = 1$
But on the otherhand we get out $\bar \omega$ values that are between $[-0.5, 0.5]$ and not $\omega_0 S_{N}$

### Code:

In [73]:
num_qubits = 3
num_energy_bits = 6
bohr_bound = 2 ** (-num_energy_bits + 1)
eps = 0.1
sigma = 5
eig_index = 2
T = 1
total_time = T
shots = 1
num_trotter_steps = 100

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)
rescaled_coeff = hamiltonian.rescaled_coeffs
# Corresponding Trotter step circuit
trotter_step_circ = trotter_step_heisenberg(num_qubits, coeffs=rescaled_coeff, symbreak=True)
hamiltonian.trotter_step_circ = trotter_step_circ

step_size = total_time / num_trotter_steps
trott_U_qt = trotter_heisenberg_qutip(num_qubits, step_size, num_trotter_steps, 
                                      coeffs=hamiltonian.rescaled_coeffs, shift=(total_time*hamiltonian.shift))

#* Initial state = eigenstate
initial_state = hamiltonian.eigenstates[:, eig_index]
initial_state = Statevector(initial_state)
print(f'Initial energy: {hamiltonian.spectrum[eig_index]}')

Original spectrum:  [-3.7    -3.177  -2.5482 -1.7     1.625   2.4547  3.2521  3.7936]
Ideal spectrum:  [0.     0.0314 0.0692 0.1201 0.3198 0.3696 0.4175 0.45  ]
Nonrescaled coefficients:  [1.  0.8 0.9 1. ]
Rescaled coefficients:  [0.06005156 0.04804125 0.05404641 0.06005156]
eXX, eYY, eZZ for qubit (0, 1)
eXX, eYY, eZZ for qubit (1, 2)
eXX, eYY, eZZ for qubit (2, 0)
Initial energy: 0.0691654616279888


In [74]:
# Trotter with 10 steps = 1 time unit of evolution with H
the_time = 1
time_evolution_trott = lambda n: np.linalg.matrix_power(trott_U_qt, 1 * n)
time_evolution = lambda t: expm(1j * t * hamiltonian.qt.full())
np.linalg.norm(time_evolution(the_time) - time_evolution_trott(the_time))

0.0002751494007943095

In [75]:
#* --- Circuit
qr_energy = QuantumRegister(num_energy_bits, name="w")
qr_sys  = QuantumRegister(num_qubits, name="sys")

cr_energy = ClassicalRegister(num_energy_bits, name="cr_w")
bithandler = BitHandler([cr_energy])

circ = QuantumCircuit(qr_energy, qr_sys, cr_energy)

# --- Initialize qregs
# Gaussian prep on energy register
if sigma != 0.:
    prep_circ = brute_prepare_gaussian_state(num_energy_bits, sigma)
    circ.compose(prep_circ, qr_energy, inplace=True)
else:  # Conventional QPE
    circ.h(qr_energy)
    
# System prep
circ.initialize(initial_state, qr_sys)
    
# --- Operator Fourier Transform of jump operator
jump_op = Operator(Pauli('X'))

#* --- OFT
trotter_step_circ = hamiltonian.trotter_step_circ
qr_energy = QuantumRegister(num_energy_bits, name='w')
qr_sys = QuantumRegister(num_qubits, name='sys')
oft_circ = QuantumCircuit(qr_energy, qr_sys, name="OFT")

# Energy before jump
initial_statevector = Statevector(circ).data
padded_rescaled_hamiltonian = np.kron(hamiltonian.qt.full(), np.eye(2**num_energy_bits))  # top-bottom = right-left
energy_before_jump = initial_statevector.conj().T @ padded_rescaled_hamiltonian @ initial_statevector
print(f'Energy before jump: {energy_before_jump.real}')

# Time evolutions for unit time, T = 1 (in H's units and scale)
U_pos = ham_evol(num_qubits, trotter_step=trotter_step_circ, num_trotter_steps=num_trotter_steps, time=total_time)
U_neg = ham_evol(num_qubits, trotter_step=trotter_step_circ, num_trotter_steps=num_trotter_steps, time=(-1)*total_time)
cU_pos = U_pos.control(1, label='+')
cU_neg = U_neg.control(1, label='-')

# exp(-i H T) E_old
for w in range(num_energy_bits):
    # circ.p(- total_time * hamiltonian.shift * 2**w, qr_energy[w])  #!
    if w != num_energy_bits - 1:
        for _ in range(2**w):
            oft_circ.compose(cU_neg, [w, *list(qr_sys)], inplace=True)
    else:  # q = last qubit (MSB) has opposite sign
        for _ in range(2**w):
            oft_circ.compose(cU_pos, [w, *list(qr_sys)], inplace=True)

# Jump A
global random_sys_qubit
random_sys_qubit = np.random.randint(0, num_qubits)
op_circ = QuantumCircuit(1, name="A")
op_circ.append(jump_op, [0])
oft_circ.compose(op_circ, qr_sys[random_sys_qubit], inplace=True)
print(f'Jump applied to qubit {random_sys_qubit}')

# For analysis
statevector = Statevector(oft_circ).data
energy_after_jump = statevector.conj().T @ padded_rescaled_hamiltonian @ statevector
print(f'Energy after jump: {energy_after_jump.real}')
omega = energy_after_jump - energy_before_jump
print(f'Energy jump: {omega.real}')

# # exp(i H T)
for w in range(num_energy_bits):
    # circ.p(total_time * hamiltonian.shift * 2**w, qr_energy[w])  #! Shift cancel
    if w != num_energy_bits - 1:
        for _ in range(2**w):
            oft_circ.compose(cU_pos, [w, *list(qr_sys)], inplace=True)
    else:  # q = last qubit (MSB) has opposite sign
        for _ in range(2**w):
            oft_circ.compose(cU_neg, [w, *list(qr_sys)], inplace=True)
    
oft_circ.compose(QFT(num_energy_bits, inverse=True), qubits=qr_energy, inplace=True)

circ.compose(oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)

circ_state = Statevector(circ).data.reshape(2**(num_energy_bits + num_qubits), 1)

Energy before jump: 0.06916546162798878
Jump applied to qubit 1
Energy after jump: 0.10809281569774586
Energy jump: 0.03892735406975707


In [76]:
# Project into energy
phase_bits = '111011'
# phase = int(phase_bits[1:], 2)

if len(phase_bits) != num_energy_bits:
    raise ValueError('This is not the right amount of phase bits')
if phase_bits[0] == '1':
    phase = (int(phase_bits[1:], 2) - 2**(num_energy_bits - 1)) # exp(i 2pi phase)
else:
    phase = int(phase_bits[1:], 2)

energy = 2 * np.pi * phase / 2**num_energy_bits
print(f'Energy: {energy}')

projector_to_energy = lambda bitstring: qt.tensor([qt.qeye(2**num_qubits), 
                                                   qt.Qobj(energy_state_for_bits(bitstring)).dag()]).full()
projected_circ_state_w = projector_to_energy(phase_bits) @ circ_state
projected_circ_state_w /= np.linalg.norm(projected_circ_state_w)

projector = lambda bitstring: qt.tensor([qt.qeye(2**num_qubits), 
                                         qt.Qobj(energy_state_for_bits(bitstring) @ energy_state_for_bits(bitstring).conj().T)]).full()
projected_full_state = projector(phase_bits) @ circ_state
projected_full_state /= np.linalg.norm(projected_full_state)



Energy: -0.4908738521234052


### Maths:

In [77]:

#* Units, labels
N = 2**(num_energy_bits)
N_labels = np.arange(N / 2, dtype=int)
N_labels_neg = np.arange(- N / 2, 0, dtype=int)
N_labels = np.concatenate((N_labels, N_labels_neg))
# N_labels_decimal_order = np.arange(- N / 2, N / 2, dtype=int)

energy_labels = 2 * np.pi * N_labels / N
time_labels = N_labels

# energy_labels_decimal_order = N_labels_decimal_order / N
# time_labels_decimal_order = N_labels_decimal_order

gauss = lambda t: np.exp(-t**2 / (4 * sigma**2))
gauss_values = gauss(time_labels)
normalized_gauss_values_bin_order = gauss_values / np.sqrt(np.sum(gauss_values**2))
print(f'Normalized gauss sum {np.linalg.norm(normalized_gauss_values_bin_order)}')

# plot gauss
# plt.plot(time_labels_decimal_order, normalized_gauss_values_bin_order)

Normalized gauss sum 1.0


In [78]:
#* Trotterized OFT
jump_op = qt.tensor([qt.qeye(2), qt.sigmax(), qt.qeye(2)]).full()
oft_op = np.zeros_like(jump_op, dtype=np.complex128)
time_evolution = lambda n: np.linalg.matrix_power(trott_U_qt.full(), n)
for i, n in enumerate(N_labels):
    oft_op += (np.exp(-1j * phase * n * 2 * np.pi  / N)  #!
                * normalized_gauss_values_bin_order[i] * time_evolution(n) @ jump_op @ time_evolution(-n)) / np.sqrt(N)
    
state_we_should_get = oft_op @ initial_state.data
state_we_should_get.reshape(2**num_qubits, 1)
state_we_should_get /= np.linalg.norm(state_we_should_get)

full_state_we_should_get = qt.tensor([
    qt.Qobj(state_we_should_get),
    qt.Qobj(energy_state_for_bits(phase_bits))
]).full()
full_state_we_should_get /= np.linalg.norm(full_state_we_should_get)

print(full_state_we_should_get.shape)
print(projected_full_state.shape)

print(state_we_should_get.shape)
print(projected_circ_state_w.shape)

#* Exact
# hamiltonian is not None and trotter is None:  
# time_evolution = lambda t: expm(1j * t * hamiltonian)
# for n in N_labels:
#     oft_op += np.exp(-1j * energy * 2 * np.pi * n) * gauss(n) * time_evolution(2 * np.pi * n) @ jump_op @ time_evolution(-2 * np.pi * n)


(512, 1)
(512, 1)
(8,)
(8, 1)


In [94]:
time_evolution = lambda n: np.linalg.matrix_power(trott_U_qt.full(), n)
trotter_time_evolution = time_evolution(1)
exact_time_evolution = expm(1j * 1 * hamiltonian.qt.full())
trott_circ_op = Operator(ham_evol(num_qubits, trotter_step=trotter_step_circ, num_trotter_steps=num_trotter_steps, time=1)).data
qt.Qobj(trott_circ_op - trotter_time_evolution).norm('fro')

0.6271585065591244

In [79]:
# Distance
fid = state_fidelity(projected_circ_state_w, state_we_should_get)
print(f'Fidelity: {fid}')
trdist_qt = qt.tracedist(qt.Qobj(projected_circ_state_w), qt.Qobj(state_we_should_get))
print(f'Trace distance: {trdist_qt}')
fid_qt = qt.fidelity(qt.Qobj(projected_circ_state_w), qt.Qobj(state_we_should_get))
print(f'Fidelity qutip: {fid_qt}')

Fidelity: 0.9536945524564571
Trace distance: 0.21518889933085444
Fidelity qutip: 0.9765728608027447


In [80]:
fid_full = state_fidelity(projected_full_state, full_state_we_should_get)
print(f'Fidelity full: {fid_full}')
trdist_qt_full = qt.tracedist(qt.Qobj(projected_full_state), qt.Qobj(full_state_we_should_get))
print(f'Trace distance full: {trdist_qt_full}')
fid_qt_full = qt.fidelity(qt.Qobj(projected_full_state), qt.Qobj(full_state_we_should_get))
print(f'Fidelity qutip full: {fid_qt_full}')


Fidelity full: 0.9536945524564571
Trace distance full: 0.21518889931669138
Fidelity qutip full: 0.9765728608027447
